In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [4]:
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in codecs.open(os.path.join(self.dirname, fname)) :
                yield line.split()

In [5]:
sentences = MySentences('/home/bahbbc/Documents/teste-personalidade') # a memory-friendly iterator

In [6]:
model = gensim.models.Word2Vec.load('/home/bahbbc/Documents/teste-personalidade/personality_1')
#model = Word2Vec.load_word2vec_format('/home/bahbbc/Documents/teste-personalidade/personality_1',binary=True)
model.init_sims(replace=True)

2017-01-30 23:38:26,377 : INFO : loading Word2Vec object from /home/bahbbc/Documents/teste-personalidade/personality_1
2017-01-30 23:38:26,568 : INFO : loading wv recursively from /home/bahbbc/Documents/teste-personalidade/personality_1.wv.* with mmap=None
2017-01-30 23:38:26,569 : INFO : setting ignored attribute syn0norm to None
2017-01-30 23:38:26,569 : INFO : setting ignored attribute cum_table to None
2017-01-30 23:38:26,570 : INFO : loaded /home/bahbbc/Documents/teste-personalidade/personality_1
2017-01-30 23:38:26,652 : INFO : precomputing L2-norms of word weight vectors


### Verify model with personality

In [7]:
df = pd.read_csv('~/personality-normalized-word2vec.csv', encoding='utf_8')
df = df.dropna()
df['isExtr'].apply(lambda x: len(x.split(' '))).sum()

3186

In [8]:
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)

### Evaluation code

In [9]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [17]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train_data["formatted_text"]:
    clean_train_reviews.append( review ) #review_to_wordlist( review, remove_stopwords=False ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test_data["formatted_text"]:
    clean_test_reviews.append( review ) # review_to_wordlist( review, remove_stopwords=False ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

NameError: name 'num_features' is not defined

In [15]:
train_data.s

KeyError: 1